<a href="https://colab.research.google.com/github/DanielAdif/Machine-Performance-Evaluation/blob/main/Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Important Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
pd. set_option('display.max_rows', 500)

#Sci-kit Learn Library
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron

#Deep Learning Library
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.layers import Dropout

In [ ]:
df = pd.read_excel('Clustered.xlsx')
df = df.set_index('Obs')
df.head()

,Number of Defect,Std. Dev Processing,%Completion,Perfectly Cut,Defect Type,Cluster
Obs,,,,,,
1,3,2.645751,108,0,1,1
2,5,1.414214,99,0,1,1
3,3,1.527525,109,0,1,1
4,3,1.732051,101,0,1,1
5,9,3.055050,75,1,1,2


In [ ]:
X = df.drop('Cluster', axis = 1)
y = df['Cluster']

In [ ]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
model = Sequential()
model.add(Dense(500, input_dim=5, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.3)

In [ ]:
model.fit(X_train, y_train, batch_size=10, epochs=300)

Epoch 1/300
26/26 [==============================] - 1s 2ms/step - loss: 1.0691 - accuracy: 0.5195
Epoch 2/300
26/26 [==============================] - 0s 2ms/step - loss: 0.5792 - accuracy: 0.7539
Epoch 3/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4283 - accuracy: 0.8281
Epoch 4/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4401 - accuracy: 0.8086
Epoch 5/300
26/26 [==============================] - 0s 2ms/step - loss: 0.3442 - accuracy: 0.8516
Epoch 6/300
26/26 [==============================] - 0s 2ms/step - loss: 0.2529 - accuracy: 0.8984
Epoch 7/300
26/26 [==============================] - 0s 2ms/step - loss: 0.2857 - accuracy: 0.8633
Epoch 8/300
26/26 [==============================] - 0s 2ms/step - loss: 0.2392 - accuracy: 0.9219
Epoch 9/300
26/26 [==============================] - 0s 3ms/step - loss: 0.2543 - accuracy: 0.9102
Epoch 10/300
26/26 [==============================] - 0s 3ms/step - loss: 0.1802 - accuracy: 0.9336
Epoch 11/

In [ ]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 0.0717 - accuracy: 0.9820


[0.07166693359613419, 0.9819819927215576]

In [ ]:
X_new = [[3, 0.7, 100, 0, 0]]
y_proba = model.predict(X_new)
y_proba.round(2)

array([[0., 1., 0.]], dtype=float32)

In [ ]:
y_pred = model.predict(X_new)
classes_x=np.argmax(y_pred,axis=1)
classes_x

array([1])

In [ ]:
from sklearn.model_selection import GridSearchCV
def baseline_model():
  model = Sequential()
  model.add(Dense(500, input_dim=5, activation='relu'))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dropout(rate = 0.1))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=4, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)

In [ ]:
results_keras = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results_keras.mean()*100, results_keras.std()*100))

Baseline: 94.27% (4.32%)


In [ ]:
estimator.fit(X_train, y_train)

In [ ]:
X_new = X_test[:3]
y_proba = estimator.predict(X_new)
y_proba.round(2)

array([1, 0, 2])

In [ ]:
y_pred = estimator.predict(X_new)
y_test

array([1, 0, 2])

In [ ]:
y_test[:3]

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)